In [1]:
import polars as pl
import pathlib
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'plotly_mimetype+notebook'
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [2]:
path_data = pathlib.Path.cwd().parent / 'data'
path_input = path_data / 'input'
path_output = path_data / 'output'

path_df = path_output / 'HY1999-2016_20230922_P2A1.csv'

In [3]:
df = pl.read_csv(str(path_df), infer_schema_length=6000)
df.head()

HarvestYear,ID2,Longitude,Latitude,SampleID,Crop,GrainSampleArea_P1,GrainSampleArea_P1_qcApplied,GrainSampleArea_P1_qcResult,GrainSampleArea_P1_qcPhrase,GrainMassWet_P1,GrainMassWet_P1_qcApplied,GrainMassWet_P1_qcResult,GrainMassWet_P1_qcPhrase,GrainMassWetInGrainSample_P1,GrainMassWetInGrainSample_P1_qcApplied,GrainMassWetInGrainSample_P1_qcResult,GrainMassWetInGrainSample_P1_qcPhrase,GrainMassWetInGrainSample_P2,GrainMassAirDryInGrainSample_P1,GrainMassAirDryInGrainSample_P1_qcApplied,GrainMassAirDryInGrainSample_P1_qcResult,GrainMassAirDryInGrainSample_P1_qcPhrase,GrainMassAirDryInGrainSample_P2,GrainMassAirDry_P1,GrainMassAirDry_P1_qcApplied,GrainMassAirDry_P1_qcResult,GrainMassAirDry_P1_qcPhrase,GrainMassDry0_P2,GrainMoistureProportionPartial_P2,GrainMoisture_P1,GrainMoisture_P1_qcApplied,GrainMoisture_P1_qcResult,GrainMoisture_P1_qcPhrase,GrainProtein_P1,GrainProtein_P1_qcApplied,GrainProtein_P1_qcResult,…,GrainMassWetInBiomassSample_P1_qcApplied,GrainMassWetInBiomassSample_P1_qcResult,GrainMassWetInBiomassSample_P1_qcPhrase,GrainMassAirDryInBiomassSample_P1,GrainMassAirDryInBiomassSample_P1_qcApplied,GrainMassAirDryInBiomassSample_P1_qcResult,GrainMassAirDryInBiomassSample_P1_qcPhrase,GrainMassAirDryInBiomassSample_P2,ResidueMassWetSubsample_P1,ResidueMassWetSubsample_P1_qcApplied,ResidueMassWetSubsample_P1_qcResult,ResidueMassWetSubsample_P1_qcPhrase,ResidueMassAirDrySubsample_P1,ResidueMassAirDrySubsample_P1_qcApplied,ResidueMassAirDrySubsample_P1_qcResult,ResidueMassAirDrySubsample_P1_qcPhrase,ResidueMoistureProportionPartialSubsample_P2,ResidueCarbon_P1,ResidueCarbon_P1_qcApplied,ResidueCarbon_P1_qcResult,ResidueCarbon_P1_qcPhrase,ResidueNitrogen_P1,ResidueNitrogen_P1_qcApplied,ResidueNitrogen_P1_qcResult,ResidueNitrogen_P1_qcPhrase,ResidueSulfur_P1,ResidueSulfur_P1_qcApplied,ResidueSulfur_P1_qcResult,ResidueSulfur_P1_qcPhrase,Comments,GrainYieldWet_P2,GrainYieldAirDry_P2,GrainYieldDry0_P2,ResidueMassWet_P2,ResidueMassAirDry_P2,ResidueMassWetPerArea_P2,ResidueMassAirDryPerArea_P2
i64,i64,f64,f64,str,str,f64,i64,i64,str,f64,i64,i64,str,f64,i64,i64,str,f64,f64,i64,i64,str,f64,f64,i64,i64,str,f64,f64,f64,i64,i64,str,f64,i64,i64,…,i64,i64,str,f64,i64,i64,str,f64,f64,i64,i64,str,f64,i64,i64,str,f64,f64,i64,i64,str,f64,i64,i64,str,f64,i64,i64,str,str,f64,f64,f64,f64,f64,f64,f64
1999,1,-117.087514,46.778729,"""1999_1""","""SW""",0.0,1,0,"""""",465.5,1,0,"""""",0.0,1,0,"""""",null,0.0,1,0,"""""",null,448.74,1,0,"""""",null,0.036004,null,1,0,"""""",null,1,0,…,1,0,"""""",448.74,1,0,"""""",null,71.9,1,0,"""""",67.9,1,0,"""""",0.055633,44.1,1,0,"""""",0.44,1,0,"""""",0.06,1,0,"""""",null,229.084646,220.836614,null,690.2,651.802225,339.665354,320.768812
1999,2,-117.087065,46.778692,"""1999_2""","""SW""",0.0,1,0,"""""",511.9,1,0,"""""",0.0,1,0,"""""",null,0.0,1,0,"""""",null,493.24,1,0,"""""",null,0.036452,null,1,0,"""""",null,1,0,…,1,0,"""""",493.24,1,0,"""""",null,106.1,1,0,"""""",100.1,1,0,"""""",0.05655,43.3,1,0,"""""",0.45,1,0,"""""",0.07,1,0,"""""",null,251.919291,242.73622,null,843.2,795.516682,414.96063,391.49443
1999,3,-117.086678,46.778791,"""1999_3""","""SW""",0.0,1,0,"""""",542.1,1,0,"""""",0.0,1,0,"""""",null,0.0,1,0,"""""",null,518.86,1,0,"""""",null,0.04287,null,1,0,"""""",null,1,0,…,1,0,"""""",518.86,1,0,"""""",null,96.3,1,0,"""""",91.7,1,0,"""""",0.047767,43.38,1,0,"""""",0.41,1,0,"""""",0.08,1,0,"""""",null,266.781496,255.344488,null,781.8,744.455452,384.744094,366.365872
1999,4,-117.08626,46.778761,"""1999_4""","""SW""",0.0,1,0,"""""",610.0,1,0,"""""",0.0,1,0,"""""",null,0.0,1,0,"""""",null,587.61,1,0,"""""",null,0.036705,null,1,0,"""""",null,1,0,…,1,0,"""""",587.61,1,0,"""""",null,109.9,1,0,"""""",104.5,1,0,"""""",0.049136,43.41,1,0,"""""",0.29,1,0,"""""",0.06,1,0,"""""",null,300.19685,289.17815,null,873.8,830.865332,430.019685,408.890419
1999,5,-117.085842,46.778666,"""1999_5""","""SW""",0.0,1,0,"""""",718.1,1,0,"""""",0.0,1,0,"""""",null,0.0,1,0,"""""",null,693.97,1,0,"""""",null,0.033603,null,1,0,"""""",nul

In [4]:
non_metric_cols = ['HarvestYear', 'ID2', 'Longitude', 'Latitude', 'SampleID', 'Crop']
metric_cols = [x for x in df.columns if x not in non_metric_cols]

df_agg_yr = (
    df.groupby(['HarvestYear', 'Crop'])
    .agg(
        [pl.col(f'{c}').mean().alias(f'{c}_mean') for c in metric_cols]
    )
    .sort(['Crop', 'HarvestYear'])
) 

print(df_agg_yr)

shape: (102, 112)
┌───────────┬──────┬────────────┬────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ HarvestYe ┆ Crop ┆ GrainSampl ┆ GrainSampl ┆ … ┆ ResidueMas ┆ ResidueMas ┆ ResidueMas ┆ ResidueMas │
│ ar        ┆ ---  ┆ eArea_P1_m ┆ eArea_P1_q ┆   ┆ sWet_P2_me ┆ sAirDry_P2 ┆ sWetPerAre ┆ sAirDryPer │
│ ---       ┆ str  ┆ ean        ┆ cApplied_m ┆   ┆ an         ┆ _mean      ┆ a_P2_mean  ┆ Area_P2_me │
│ i64       ┆      ┆ ---        ┆ ean        ┆   ┆ ---        ┆ ---        ┆ ---        ┆ an         │
│           ┆      ┆ f64        ┆ ---        ┆   ┆ f64        ┆ f64        ┆ f64        ┆ ---        │
│           ┆      ┆            ┆ f64        ┆   ┆            ┆            ┆            ┆ f64        │
╞═══════════╪══════╪════════════╪════════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 2001      ┆ null ┆ null       ┆ 1.0        ┆ … ┆ null       ┆ null       ┆ null       ┆ null       │
│ 2003      ┆ null ┆ null       ┆ 1.0        ┆ … ┆ null

In [9]:
def create_yield_comparison_plot(df, crop, grain_or_residue):

    # init with 'grain' values
    wet_var = 'GrainYieldWet_P2'
    air_var = 'GrainYieldAirDry_P2'
    zero_var = 'GrainYieldDry0_P2'

    if(grain_or_residue == 'residue'):
        wet_var = 'ResidueMassWetPerArea_P2'
        air_var = 'ResidueMassAirDryPerArea_P2'
        zero_var = 'ResidueMassDry0PerArea_P2'
        

    fig = go.Figure()

    crop_df = df.filter(pl.col('Crop') == crop)

    x = crop_df['HarvestYear'].to_list()

    # Start/End cutoffs for years were methods and/or data were different
    cutoff_a0 = 1999
    cutoff_a1 = 2010

    cutoff_b0 = 2011
    cutoff_b1 = 2012

    cutoff_c0 = 2013
    cutoff_c1 = 2016

    # Wet averages
    wet_mean_1999_2010 = crop_df.filter(pl.col('HarvestYear') <= cutoff_a1)[wet_var].mean()
    wet_mean_2011_2012 = crop_df.filter((pl.col('HarvestYear') > cutoff_a1) & (pl.col('HarvestYear') <= cutoff_b1))[wet_var].mean()
    wet_mean_2013_2016 = crop_df.filter(pl.col('HarvestYear') > cutoff_b1)[wet_var].mean()

    # Air dry averages
    air_mean_1999_2010 = crop_df.filter(pl.col('HarvestYear') <= cutoff_a1)[air_var].mean()
    air_mean_2011_2012 = crop_df.filter((pl.col('HarvestYear') > cutoff_a1) & (pl.col('HarvestYear') <= cutoff_b1))[air_var].mean()
    air_mean_2013_2016 = crop_df.filter(pl.col('HarvestYear') > cutoff_b1)[air_var].mean()

    # Oven dry averages
    oven_mean_1999_2010 = crop_df.filter(pl.col('HarvestYear') <= cutoff_a1)[zero_var].mean()
    oven_mean_2011_2012 = crop_df.filter((pl.col('HarvestYear') > cutoff_a1) & (pl.col('HarvestYear') <= cutoff_b1))[zero_var].mean()
    oven_mean_2013_2016 = crop_df.filter(pl.col('HarvestYear') > cutoff_b1)[zero_var].mean()

    # Box plots
    fig.add_trace(go.Box(
        x = x,
        y = crop_df[wet_var].to_list(),
        name = 'Wet',
        marker_color='blue'
    ))

    fig.add_trace(go.Box(
        x = x,
        y = crop_df[air_var].to_list(),
        name='Air',
        marker_color='green'
    ))

    fig.add_trace(go.Box(
        x = x,
        y = crop_df[zero_var].to_list(),
        name='Zero',
        marker_color = 'red'
    ))

    # Title, xaxis, add dashed v lines
    fig.update_layout(
        boxmode='group',
        title={'text':crop,
               'y':0.9,
               'x':0.5,
               'xanchor':'center',
               'yanchor':'top'},
        xaxis=dict(range=[1998,2017]))
    fig.add_vline(x = cutoff_a1 + 0.5, line_width = 3, line_dash='dash')
    fig.add_vline(x = cutoff_b1 + 0.5, line_width = 3, line_dash='dash')

    #Wet averages
    fig.add_shape(type='line',
                  x0=cutoff_a0,
                  x1=cutoff_a1,
                  y0=wet_mean_1999_2010,
                  y1=wet_mean_1999_2010,
                  xref='x',
                  yref='y',
                  line = dict(color='Blue'))
    fig.add_shape(type='line',
                  x0=cutoff_b0,
                  x1=cutoff_b1,
                  y0=wet_mean_2011_2012,
                  y1=wet_mean_2011_2012,
                  xref='x',
                  yref='y',
                  line = dict(color='Blue'))
    fig.add_shape(type='line',
                  x0=cutoff_c0,
                  x1=cutoff_c1,
                  y0=wet_mean_2013_2016,
                  y1=wet_mean_2013_2016,
                  xref='x',
                  yref='y',
                  line = dict(color='Blue'))
    
    #Wet averages
    fig.add_shape(type='line',
                  x0=cutoff_a0,
                  x1=cutoff_a1,
                  y0=air_mean_1999_2010,
                  y1=air_mean_1999_2010,
                  xref='x',
                  yref='y',
                  line = dict(color='Green'))
    fig.add_shape(type='line',
                  x0=cutoff_b0,
                  x1=cutoff_b1,
                  y0=air_mean_2011_2012,
                  y1=air_mean_2011_2012,
                  xref='x',
                  yref='y',
                  line = dict(color='Green'))
    fig.add_shape(type='line',
                  x0=cutoff_c0,
                  x1=cutoff_c1,
                  y0=air_mean_2013_2016,
                  y1=air_mean_2013_2016,
                  xref='x',
                  yref='y',
                  line = dict(color='Green'))
    
    #NIR-Adjusted (Zero) averages lines
    fig.add_shape(type='line',
                  x0=cutoff_a0,
                  x1=cutoff_a1,
                  y0=oven_mean_1999_2010,
                  y1=oven_mean_1999_2010,
                  xref='x',
                  yref='y',
                  line = dict(color='Red'))
    fig.add_shape(type='line',
                  x0=cutoff_b0,
                  x1=cutoff_b1,
                  y0=oven_mean_2011_2012,
                  y1=oven_mean_2011_2012,
                  xref='x',
                  yref='y',
                  line = dict(color='Red'))
    fig.add_shape(type='line',
                  x0=cutoff_c0,
                  x1=cutoff_c1,
                  y0=oven_mean_2013_2016,
                  y1=oven_mean_2013_2016,
                  xref='x',
                  yref='y',
                  line = dict(color='Red'))

    fig.show()

In [10]:
crops = list(set(df['Crop'].to_list()))

# Box plots

The following figures show box plots of each crop across the years. Vertical dashed lines indicate a change of methods (and/or available data). The horizontal lines indicate mean values for the years within the vertical lines.

## Grain yields (g/m2)

In [ ]:
for crop in crops:
    create_yield_comparison_plot(df, crop, 'grain')

In [20]:
#harvest_2013_2016_calc = (harvest_2013_2016
#        .with_columns(
#            (pl.col('BiomassAirDry_P1') - pl.col('GrainMassAirDry_P1'))
#            .alias('ResidueMassAirDry_P2')
#        )
#        .with_columns(
#            pl.when(pl.col('BiomassSampleArea_P1') > 0)
#            .then(pl.col('ResidueMassAirDry_P2') / pl.col('BiomassSampleArea_P1'))
#            .otherwise(None)
#            .alias('ResidueMassAirDryPerArea_P2')
#        )


grainAirDry_qcResult_cols = ['GrainMassWet_P1_qcResult', 'GrainMassWetInGrainSample_P1_qcResult', 'GrainMassAirDryInGrainSample_P1_qcResult', 'GrainMassAirDry_P1_qcResult', 'GrainMassWetInBiomassSample_P1_qcResult','GrainMassAirDryInBiomassSample_P1_qcResult']
grainDry0_qcResult_cols = ['GrainMassWet_P1_qcResult', 'GrainMassWetInGrainSample_P1_qcResult', 'GrainMassAirDryInGrainSample_P1_qcResult', 'GrainMassAirDry_P1_qcResult', 'GrainMoisture_P1_qcResult', 'GrainMassWetInBiomassSample_P1_qcResult','GrainMassAirDryInBiomassSample_P1_qcResult']

clean_df = (df
    .with_columns(
        pl.sum([pl.col(i) for i in grainAirDry_qcResult_cols])
        .alias('GrainAirDryVars_qcResultSum')
    )
    .with_columns(
        pl.sum([pl.col(i) for i in grainDry0_qcResult_cols])
        .alias('GrainDry0Vars_qcResultSum')
    )
    .with_columns(
        pl.when(pl.col('GrainAirDryVars_qcResultSum') > 0)
        .then(None)
        .otherwise(pl.col('GrainYieldAirDry_P2'))
        .alias('GrainYieldAirDry_P2')
    )
    .with_columns(
        pl.when(pl.col('GrainDry0Vars_qcResultSum') > 0)
        .then(None)
        .otherwise(pl.col('GrainYieldDry0_P2'))
        .alias('GrainYieldDry0_P2')
    ))

print('null in clean: ' + str(clean_df['GrainYieldAirDry_P2'].null_count()) + ' vs nulls in original: ' + str(df['GrainYieldAirDry_P2'].null_count()))

null in clean: 1380 vs nulls in original: 1056


In [21]:

for crop in crops:
    create_yield_comparison_plot(clean_df, crop, 'grain')

## Residue yields (g/m2)

In [8]:
for crop in crops:
    create_yield_comparison_plot(crop, 'residue')

ColumnNotFoundError: ResidueMassDry0PerArea_P2